

---

# Part 3: The Network and Transport Layers – Addressing, Routing, and Reliable Delivery

## Chapter 7: The Art of Subnetting and CIDR

In Chapter 6, you learned the structure of an IPv4 address and the crucial role of the subnet mask in separating the network and host portions. You now understand that an IP address like `192.168.1.105` with a mask of `255.255.255.0` belongs to the network `192.168.1.0/24`. But this raises a practical question: is a `/24` network always the right size? What if you have a small office with only ten devices? Using a `/24` network wastes over 200 IP addresses. What if you are a large university with thousands of devices? A single `/24` is far too small.

This is where **subnetting** comes in. Subnetting is the art and science of taking a larger block of IP addresses and dividing it into smaller, more efficient, and manageable pieces called **subnets**. It is one of the most fundamental and essential skills for any network professional. This chapter will guide you from the basic concepts through advanced techniques like VLSM and route summarization.

By the end of this chapter, you will be able to look at any IP address and subnet mask and instantly determine the network address, broadcast address, and usable host range. You will be able to design an addressing scheme for any size network.

### 7.1 Why Subnet? Efficient IP Address Management

Before we dive into the "how," it is critical to understand the "why." Subnetting is not just an academic exercise; it solves several real-world problems:

1.  **Efficient Use of Address Space:** As we saw in Chapter 6, IPv4 address space is a finite and valuable resource. Assigning a `/24` network (254 usable addresses) to a point-of-sale terminal network with only 15 devices would waste over 200 addresses. Subnetting allows you to allocate only the number of addresses you actually need, preserving the rest for future use.

2.  **Improved Network Performance:** Subnetting creates smaller broadcast domains. A broadcast in a large, flat network must be processed by every device. By dividing that large network into smaller subnets, you contain broadcasts within each subnet, reducing overall network traffic and improving performance for all devices.

3.  **Enhanced Security:** Subnetting allows you to isolate different types of traffic and users. You can place all human resources devices in one subnet, all engineering devices in another, and all guest devices in a third. You can then place a firewall or router with access control lists between these subnets to control who can talk to whom. This is a fundamental security principle called **segmentation**.

4.  **Simplified Management:** Smaller subnets are easier to troubleshoot and manage. Problems are contained within a subnet and are less likely to affect the entire organization. It also makes it easier to apply different policies (like Quality of Service or security rules) to different groups of users.

### 7.2 Classful Addressing (A, B, C) and Its Limitations

In the early days of the Internet, before subnetting became widespread, IP addresses were allocated based on a system called **classful addressing**. This system divided the IP address space into fixed-size classes, each with a predefined default subnet mask.

- **Class A:** For very large networks. The first octet identified the network (0-127), with a default mask of `255.0.0.0` or `/8`. This provided 126 networks, each with over 16 million host addresses.
- **Class B:** For medium-sized networks. The first two octets identified the network (128-191), with a default mask of `255.255.0.0` or `/16`. This provided around 16,000 networks, each with 65,534 host addresses.
- **Class C:** For small networks. The first three octets identified the network (192-223), with a default mask of `255.255.255.0` or `/24`. This provided over 2 million networks, each with 254 host addresses.
- **Class D and E:** Reserved for multicast and experimental use, respectively.

This system was simple but incredibly wasteful. If a company needed 300 IP addresses, they could not use a Class C (only 254 addresses), so they would be forced to request a Class B, which provided 65,534 addresses—an enormous waste. This rigid, wasteful system was a major driver of IPv4 address exhaustion and led to the development of classless addressing.

### 7.3 Classless Inter-Domain Routing (CIDR): The `/` Notation

**Classless Inter-Domain Routing (CIDR)** , introduced in 1993 (RFC 1517), replaced the wasteful classful system. The fundamental change was to decouple the IP address from its default mask. With CIDR, you can use *any* subnet mask, not just the classful ones. This allows you to carve up address space with much finer granularity.

CIDR introduced a new, more concise way to write subnet masks: the **slash notation** (or CIDR notation). Instead of writing out `255.255.255.0`, you simply write `/24` after the IP address. The number after the slash is the number of consecutive 1-bits in the subnet mask.

For example:
- `/24` means 24 bits of network, which is `11111111.11111111.11111111.00000000` = `255.255.255.0`
- `/25` means 25 bits of network, which is `11111111.11111111.11111111.10000000` = `255.255.255.128`
- `/26` means 26 bits of network, which is `11111111.11111111.11111111.11000000` = `255.255.255.192`

This notation is now the universal standard. You will always see IP addresses written with their CIDR mask, such as `192.168.1.0/24` or `10.10.10.0/27`.

### 7.4 Subnetting a Network: Borrowing Bits

At its core, subnetting is the process of **borrowing bits** from the host portion of an address and adding them to the network portion. These borrowed bits become **subnet bits**, which are used to identify the individual subnets within the larger network.

Think of it like a telephone number. The area code is the network portion, and the local number is the host portion. Subnetting is like adding an extra digit to the area code to create multiple, smaller exchanges within the same city.

Let's walk through the process step-by-step.

**Scenario:** You have been given the network `192.168.1.0/24` and you need to create **4 subnets** of equal size.

**Step 1: Determine How Many Bits to Borrow**
The formula for calculating the number of subnets is **2^n**, where **n** is the number of bits you borrow.
We need 4 subnets. So, 2^n = 4. Solving for n, we find that n = 2. Therefore, we need to borrow **2 bits** from the host portion.

**Step 2: Determine the New Subnet Mask**
Our original mask is /24. If we borrow 2 bits, our new mask becomes /24 + 2 = **/26**.
In dotted decimal, a /26 mask is `255.255.255.192`. Let's see why in binary:
- Original /24: `11111111.11111111.11111111.00000000`
- Borrow 2 bits: `11111111.11111111.11111111.**11**000000`

The last octet is now `11000000`. The first two bits of this octet (the `11`) are our new subnet bits. The remaining six bits (`000000`) are the host bits.

**Step 3: Determine the Block Size (Increment)**
The **block size** is the number of addresses in each subnet. It is determined by the value of the lowest-order network bit (the last 1-bit in the mask). A simpler way: subtract the last non-zero octet of the subnet mask from 256.

In our /26 mask (`255.255.255.192`), the last non-zero octet is 192. So, 256 - 192 = **64**. This means each subnet will have a block size of 64 addresses.

**Step 4: List the Subnets**
We start with the original network address (`192.168.1.0`) and add the block size (64) to create each subsequent subnet. We do this four times (once for each subnet we need).

- **Subnet 0:** `192.168.1.0` to `192.168.1.63`
- **Subnet 1:** `192.168.1.64` to `192.168.1.127`
- **Subnet 2:** `192.168.1.128` to `192.168.1.191`
- **Subnet 3:** `192.168.1.192` to `192.168.1.255`

**Step 5: Determine Network, Broadcast, and Usable Host Ranges**
For each subnet, we can now identify the key addresses:
- **Network Address:** The first address in the block.
- **Broadcast Address:** The last address in the block.
- **Usable Host Addresses:** All addresses between the network and broadcast addresses. The number of usable hosts is **2^h - 2**, where **h** is the number of host bits remaining. Here, we have 6 host bits (from the original 8, we borrowed 2). So, 2^6 = 64, minus 2 = **62 usable hosts per subnet**.

Let's break down Subnet 1 as an example:
- Network Address: `192.168.1.64`
- First Usable Host: `192.168.1.65`
- Last Usable Host: `192.168.1.126`
- Broadcast Address: `192.168.1.127`

You have successfully divided a single `/24` network into four `/26` subnets, each capable of supporting 62 devices.

### 7.5 Calculating Subnets, Hosts, and Ranges (with Step-by-Step Exercises)

The best way to master subnetting is through practice. Let's work through several examples.

**Exercise 1:**
You are given the network `172.16.0.0/16`. You need to create subnets that each support at least 500 hosts. How many subnets can you create, and what is the new subnet mask?

1.  **Calculate host bits needed:** We need at least 500 usable hosts. The formula for usable hosts is 2^h - 2 >= 500.
    - 2^8 - 2 = 254 (too few)
    - 2^9 - 2 = 510 (enough)
    So we need **9 host bits** (h = 9).

2.  **Calculate new subnet mask:** The original mask is /16, which means 16 network bits. IPv4 addresses have 32 total bits. If we need 9 host bits, the remaining bits must be network + subnet bits.
    - Network + Subnet bits = 32 - 9 = **23 bits**.
    - Therefore, the new subnet mask is **/23**.

3.  **Determine number of subnets:** We borrowed bits from the original /16 to get to /23. The number of bits borrowed is 23 - 16 = **7 bits**.
    - Number of subnets = 2^7 = **128 subnets**.

4.  **Determine block size:** A /23 mask is `255.255.254.0`. The last non-zero octet is 254. Block size = 256 - 254 = **2**. So each subnet increments by 2 in the third octet.

5.  **List the first few subnets:**
    - Subnet 0: `172.16.0.0/23` (Network: `172.16.0.0`, Broadcast: `172.16.1.255`)
    - Subnet 1: `172.16.2.0/23` (Network: `172.16.2.0`, Broadcast: `172.16.3.255`)
    - Subnet 2: `172.16.4.0/23` (Network: `172.16.4.0`, Broadcast: `172.16.5.255`)
    And so on.

**Exercise 2:**
You are given the network `10.0.0.0/8`. You need to create 1000 subnets. What is the new subnet mask, and how many hosts will each subnet support?

1.  **Calculate subnet bits needed:** We need 1000 subnets. The formula is 2^n >= 1000.
    - 2^9 = 512 (not enough)
    - 2^10 = 1024 (enough)
    So we need to borrow **10 subnet bits** (n = 10).

2.  **Calculate new subnet mask:** Original mask is /8. We borrow 10 bits, so the new mask is /8 + 10 = **/18**.

3.  **Calculate host bits remaining:** Total bits are 32. With a /18 mask, the number of host bits is 32 - 18 = **14 bits**.

4.  **Calculate usable hosts per subnet:** Usable hosts = 2^14 - 2 = 16384 - 2 = **16,382 hosts per subnet**.

5.  **Determine block size:** A /18 mask is `255.255.192.0`. The last non-zero octet is 192 in the third octet. Block size = 256 - 192 = **64**. So each subnet increments by 64 in the third octet.

### 7.6 VLSM (Variable Length Subnet Masks): Subnetting a Subnet for Maximum Efficiency

In all the examples so far, we have practiced **Fixed Length Subnet Masking (FLSM)** , where every subnet we create is the same size. This is simple but can still be wasteful. What if you have a point-of-sale network that needs only 2 usable IP addresses, a printer network that needs 10, and a user network that needs 200? Giving them all the same size subnet (e.g., a /24) would waste hundreds of addresses.

**VLSM (Variable Length Subnet Masks)** solves this problem by allowing you to **subnet a subnet**. You can take a larger block and divide it into smaller blocks of varying sizes, allocating only the number of addresses each network actually needs. This is the standard practice in modern network design.

**The Golden Rule of VLSM:** When subnetting with VLSM, you must always start with the largest subnet requirement first (the one needing the most hosts) and work your way down to the smallest. This prevents your subnets from overlapping.

**VLSM Example Scenario:**
You are given the network `192.168.1.0/24` and you have the following requirements:

- **LAN A:** 100 hosts
- **LAN B:** 50 hosts
- **LAN C:** 20 hosts
- **Point-to-Point Link (Router-to-Router):** 2 hosts

**Step 1: Sort by Largest Requirement (Descending)**
1.  LAN A: 100 hosts
2.  LAN B: 50 hosts
3.  LAN C: 20 hosts
4.  P2P Link: 2 hosts

**Step 2: Calculate the Subnet Mask for the Largest Requirement (LAN A)**
We need at least 100 usable hosts. Find the smallest `h` where 2^h - 2 >= 100.
- 2^6 - 2 = 62 (too few)
- 2^7 - 2 = 126 (enough)
So we need 7 host bits. This means the network portion must be 32 - 7 = 25 bits. LAN A requires a **/25** mask.
- Block size for /25: 256 - 128 = 128.
- We take the first block: **`192.168.1.0/25`**
    - Network: `192.168.1.0`
    - Usable Hosts: `192.168.1.1` to `192.168.1.126` (126 addresses, enough for 100)
    - Broadcast: `192.168.1.127`

**Step 3: Next Largest Requirement (LAN B)**
We now have the remaining address space, starting from `192.168.1.128`. We need 50 hosts.
Find the smallest `h` where 2^h - 2 >= 50.
- 2^5 - 2 = 30 (too few)
- 2^6 - 2 = 62 (enough)
We need 6 host bits. This means a /26 mask (32 - 6 = 26).
- Block size for /26: 64.
- We take the next available block starting at `192.168.1.128`: **`192.168.1.128/26`**
    - Network: `192.168.1.128`
    - Usable Hosts: `192.168.1.129` to `192.168.1.190` (62 addresses, enough for 50)
    - Broadcast: `192.168.1.191`

**Step 4: Next Largest Requirement (LAN C)**
Remaining space starts at `192.168.1.192`. We need 20 hosts.
Find the smallest `h` where 2^h - 2 >= 20.
- 2^4 - 2 = 14 (too few)
- 2^5 - 2 = 30 (enough)
We need 5 host bits. This means a /27 mask (32 - 5 = 27).
- Block size for /27: 32.
- We take the next available block starting at `192.168.1.192`: **`192.168.1.192/27`**
    - Network: `192.168.1.192`
    - Usable Hosts: `192.168.1.193` to `192.168.1.222` (30 addresses, enough for 20)
    - Broadcast: `192.168.1.223`

**Step 5: Smallest Requirement (Point-to-Point Link)**
Remaining space starts at `192.168.1.224`. We need only 2 hosts (a point-to-point link only needs two IP addresses, one for each router interface).
Find the smallest `h` where 2^h - 2 >= 2.
- 2^2 - 2 = 2 (exactly enough)
We need 2 host bits. This means a /30 mask (32 - 2 = 30). This is the standard mask for point-to-point links.
- Block size for /30: 4.
- We take the next available block starting at `192.168.1.224`: **`192.168.1.224/30`**
    - Network: `192.168.1.224`
    - Usable Hosts: `192.168.1.225` and `192.168.1.226`
    - Broadcast: `192.168.1.227`

With VLSM, we have efficiently allocated just enough addresses for each requirement and still have a small block of unused addresses starting at `192.168.1.228` for future growth. This is vastly more efficient than giving all four networks a `/24` or even all of them a `/26`.

### 7.7 Supernetting and Route Summarization

Subnetting is about taking a large network and breaking it into smaller pieces. **Supernetting** (also called **route summarization** or **aggregation**) does the opposite: it combines multiple contiguous smaller networks into a single, larger network. This is a critical technique used in Internet routing to keep the global routing tables from becoming impossibly large.

Think of it like a zip code. Instead of having a separate entry in a postal worker's route for every single house on a street, they have a single entry for the entire zip code. Supernetting works the same way for routers.

**Example:**
Imagine a company owns four contiguous /24 networks:
- `192.168.0.0/24`
- `192.168.1.0/24`
- `192.168.2.0/24`
- `192.168.3.0/24`

Instead of announcing four separate routes to the Internet, their router can announce a single summarized route: **`192.168.0.0/22`** .

How do we get a /22 from four /24s?
- `/24` has 24 network bits.
- To combine four networks (2^2 = 4), we need to "borrow" 2 bits *from the network side* to create a larger network. This is the inverse of subnetting.
- 24 - 2 = 22. So the summarized route is a `/22`.
- A `/22` mask is `255.255.252.0`. Its block size is 4 in the third octet, which perfectly covers `192.168.0.0` through `192.168.3.255`.

**Benefits of Route Summarization:**
1.  **Smaller Routing Tables:** Routers have fewer entries to process, which improves performance and reduces memory usage.
2.  **Faster Convergence:** When a specific subnet within the summarized block goes down, routers outside the block don't need to know about it. Only the router advertising the summary needs to handle the flapping route. This prevents instability from spreading across the entire Internet.
3.  **Reduced Routing Update Overhead:** Less information needs to be exchanged between routers.

Route summarization is a best practice in network design and is a core feature of routing protocols like OSPF and EIGRP.

---

### Chapter 7: Hands-On Challenge

The only way to truly learn subnetting is through practice. Here are some exercises to work through. Use pencil and paper.

1.  **Basic Subnetting:**
    You have the network `10.10.0.0/16`. You need to create 8 subnets of equal size.
    - How many bits must you borrow?
    - What is the new subnet mask in slash and dotted decimal notation?
    - What is the block size?
    - List the first 3 subnets (network address and broadcast address).
    - How many usable hosts are there per subnet?

2.  **Finding the Answers:**
    For the IP address `172.16.50.25/19`:
    - What is the subnet mask in dotted decimal?
    - What is the network address?
    - What is the broadcast address?
    - What is the first usable host address?
    - What is the last usable host address?

3.  **VLSM Design:**
    You are given the network `10.0.0.0/8`. Design a VLSM scheme for the following requirements. Start with the largest requirement.
    - **HQ LAN:** 4,000 hosts
    - **West Coast LAN:** 2,000 hosts
    - **East Coast LAN:** 1,000 hosts
    - **Sales LAN:** 500 hosts
    - **WAN Link 1:** 2 hosts
    - **WAN Link 2:** 2 hosts

    (Hint: Calculate the required mask for each, then allocate blocks sequentially.)

---

This chapter has equipped you with one of the most powerful tools in a network engineer's toolkit. You can now take a block of IP addresses and carve it up with surgical precision to meet any requirement, from a massive data center to a tiny point-to-point link.

In the next chapter, we will turn our attention to the future. While IPv4 and the skills you have just mastered remain essential, the world is running out of IPv4 addresses. We will explore **IPv6**, the next-generation Internet Protocol designed to solve this problem and provide a virtually limitless supply of addresses for the future.